<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/langgraph_essay_writter_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai" "langchain_community" "docarray" "langchain_experimental" "aiosqlite"

In [ ]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
os.environ["TAVILY_API_KEY"] = userdata.get('tavily_api_key')

# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    verbose=True
)


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

# Define a Simple Agent State;
# Agent State is accessible to all parts of the graph.
class AgentState(TypedDict):
  task: str
  plan: str
  draft: str
  critique: str
  content: list[str]
  revision_number: int
  max_revisions: int

In [ ]:
tool = TavilySearchResults(max_results=2)
#print(tool.name)

In [17]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

from pydantic import BaseModel

class Queries(BaseModel):
  queries: list[str]

In [ ]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. Write such an outine for the user provided topic. Give an outline\
of the essay along with any relevant notes or instructions for the sections."""
WRITTER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays. Generate the best essay possible for the user's request and\
the initial plan. If the user provides critique, respond with a revised version of your previous attemps. Utilize all the information below as needed:\
------
{content}"""

REFLECTION_PROMPT = """You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth and so on"""

RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can be used when writing the following essay. \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""

RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing critiques to be used when making any requested revisions (as outlined below).\
Generate a list of search queries that will gather any relevant information."""

In [30]:
# Define action nodes
def plan_node(state: AgentState):
  messages = [
      SystemMessage(content=PLAN_PROMPT),
      HumanMessage(content=state['task'])
  ]
  response = llm.invoke(messages)
  return {"plan": response.content}

def research_plan_node(state: AgentState):
  queries = llm.with_structured_output(Queries).invoke([
      SystemMessage(content=RESEARCH_PLAN_PROMPT),
      HumanMessage(content=state['task'])
  ])
  content = state['content'] or []
  for q in queries.queries:
    response = tool.invoke(q)
    for r in response:
      content.append(r['content'])
  return {"content": content}

def generation_node(state: AgentState):
  content = "\n\n".join(state['content']) or []
  user_message = HumanMessage(
      content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
  messages = [
      SystemMessage(content=WRITTER_PROMPT.format(content=content)
      ),
      user_message
      ]
  response = llm.invoke(messages)
  return {
      "draft": response.content,
      "revision_number": state.get('revision_number', 1) + 1
  }
def reflection_node(state: AgentState):
  messages = [
      SystemMessage(content=REFLECTION_PROMPT),
      HumanMessage(content=state['draft'])
  ]
  response = llm.invoke(messages)
  return {"critique": response.content}

def research_critique_node(state: AgentState):
  queries = llm.with_structured_output(Queries).invoke([
      SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
      HumanMessage(content=state['critique'])
  ])
  content = state['content'] or []
  for q in queries.queries:
    response = tool.invoke(q)
    for r in response:
      content.append(r['content'])
  return {"content": content}

In [31]:
# Define conditions
def should_continue(state):
  return "reflect" if state['revision_number'] <= state['max_revisions'] else END

In [32]:
# Define the graph
builder = StateGraph(AgentState)
builder.add_node("planner", plan_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_critique", research_critique_node)

builder.add_conditional_edges(
    "generate",
    should_continue,
    {"reflect": "reflect", END: END}
)
builder.set_entry_point("planner")
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")
builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")
graph = builder.compile(checkpointer=memory)

In [ ]:
thread = {"configurable": {"thread_id": "1"}}
for event in graph.stream({
      "task": "Write an essay about the impact of AI on human society with the next 5 years",
      "max_revisions": 2,
      "revision_number": 1,
      "content": []
    }, thread):
    print(event)